In [9]:
from tqdm import tqdm
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

google_colab = False
with_validation = True
latent_dim_gmf = 30
latent_dim_mlp = 3
batch_size = 256
epochs = 30
LR = 0.001
WEIGHT_DECAY = 1e-4

In [10]:
if google_colab:
    from google.colab import drive

    drive.mount('/content/drive')
    train_path = "/content/drive/MyDrive/DIS_recomander_sys/train.csv"
    test_path = "/content/drive/MyDrive/DIS_recomander_sys/test.csv"
else:
    train_path = "../data/train.csv"
    test_path = "../data/test.csv"

In [11]:
# Create a dataset class
class InteractionDataset(Dataset):
    def __init__(self, user_item_pairs, ratings):
        self.user_item_pairs = user_item_pairs
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.user_item_pairs[idx], self.ratings[idx]


# Define the NCF model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim_gmf, latent_dim_mlp):
        super(NCF, self).__init__()
        # Embeddings for MLP part
        self.user_embedding_mlp = nn.Embedding(num_users, latent_dim_mlp)
        self.item_embedding_mlp = nn.Embedding(num_items, latent_dim_mlp)

        # Embeddings for GMF part
        self.user_embedding_gmf = nn.Embedding(num_users, latent_dim_gmf)
        self.item_embedding_gmf = nn.Embedding(num_items, latent_dim_gmf)

        # Fully connected layers for MLP
        self.fc_layers = nn.Sequential(
            nn.Linear(latent_dim_mlp * 2, 128),
            nn.BatchNorm1d(128),  # Batch normalization
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 80),
            nn.BatchNorm1d(80),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

        # Final output layer (after concatenating GMF and MLP outputs)
        self.final_layer = nn.Linear(latent_dim_gmf + 80, 1)  # GMF (latent_dim) + MLP output (64)

    def forward(self, user, item):
        # MLP embeddings
        user_embed_mlp = self.user_embedding_mlp(user)  # Shape: [batch_size, latent_dim]
        item_embed_mlp = self.item_embedding_mlp(item)  # Shape: [batch_size, latent_dim]

        # GMF embeddings
        user_embed_gmf = self.user_embedding_gmf(user)  # Shape: [batch_size, latent_dim]
        item_embed_gmf = self.item_embedding_gmf(item)  # Shape: [batch_size, latent_dim]

        # GMF interaction (element-wise product)
        gmf_output = torch.mul(user_embed_gmf, item_embed_gmf)  # Shape: [batch_size, latent_dim]

        # MLP interaction (concatenation)
        mlp_input = torch.cat([user_embed_mlp, item_embed_mlp], dim=-1)  # Shape: [batch_size, latent_dim * 2]
        mlp_output = self.fc_layers(mlp_input)  # Shape: [batch_size, 64]

        # Concatenate GMF and MLP outputs
        combined = torch.cat([gmf_output, mlp_output], dim=-1)  # Shape: [batch_size, latent_dim + 64]

        # Final prediction layer
        output = self.final_layer(combined).squeeze()  # Shape: [batch_size]

        return output


In [12]:
# Prepare data
train_df = pd.read_csv(train_path)
user_item_pairs = train_df[["user_id", "book_id"]].values
#Map the user_id and book_id to a unique index
user_to_index = {user_id: idx for idx, user_id in enumerate(train_df['user_id'].unique())}
item_to_index = {book_id: idx for idx, book_id in enumerate(train_df['book_id'].unique())}
train_df['user_idx'] = train_df['user_id'].apply(lambda x: user_to_index[x])
train_df['book_idx'] = train_df['book_id'].apply(lambda x: item_to_index[x])
user_item_index_pairs = train_df[["user_idx", "book_idx"]].values
ratings = train_df["rating"].values

# Define constants
num_users = len(user_to_index)
num_items = len(item_to_index)
print(num_users, num_items)

# DataLoader
dataset = InteractionDataset(user_item_index_pairs, ratings)

# Split into training and validation sets (80-20 split)
if with_validation:
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
else:
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_loader = None

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NCF(num_users, num_items, latent_dim_gmf, latent_dim_mlp).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

18905 15712


In [13]:
if google_colab:
    print(torch.cuda.is_available())
    print(torch.cuda.memory_allocated())
    print(torch.cuda.memory_reserved())

In [14]:
for epoch in tqdm(range(epochs), desc="Epochs", unit="epoch"):
    model.train()
    total_train_loss = 0

    # Training loop
    for user_item, rating in train_loader:
        user, item = user_item[:, 0].long().to(device), user_item[:, 1].long().to(device)
        rating = rating.float().to(device)

        optimizer.zero_grad()
        predictions = model(user, item)
        loss = criterion(predictions, rating)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    # Validation loop
    if with_validation:
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for user_item, rating in val_loader:
                user, item = user_item[:, 0].long().to(device), user_item[:, 1].long().to(device)
                rating = rating.float().to(device)
                predictions = model(user, item)
                loss = criterion(predictions, rating)
                total_val_loss += loss.item()
        print(
            f"Epoch {epoch + 1}, Train Loss: {total_train_loss / len(train_loader):.4f}, Val Loss: {total_val_loss / len(val_loader):.4f}")
    else:
        print(f"Epoch {epoch + 1}, Train Loss: {total_train_loss / len(train_loader):.4f}")


Epochs:   3%|▎         | 1/30 [00:03<01:44,  3.59s/epoch]

Epoch 1, Train Loss: 2.0643, Val Loss: 1.4776


Epochs:   7%|▋         | 2/30 [00:06<01:31,  3.27s/epoch]

Epoch 2, Train Loss: 1.4483, Val Loss: 1.3356


Epochs:  10%|█         | 3/30 [00:09<01:26,  3.21s/epoch]

...

Epoch 28, Train Loss: 0.4774, Val Loss: 0.7521


Epochs:  97%|█████████▋| 29/30 [01:30<00:03,  3.07s/epoch]

Epoch 29, Train Loss: 0.4657, Val Loss: 0.7486


Epochs: 100%|██████████| 30/30 [01:34<00:00,  3.14s/epoch]

Epoch 30, Train Loss: 0.4540, Val Loss: 0.7441


In [15]:
test_df = pd.read_csv(test_path)


# Imputation (predict missing values)
def predict_missing_values(model, test_df, user_to_index, item_to_index):
    test_df['user_idx'] = test_df['user_id'].apply(lambda x: user_to_index[x])
    test_df['book_idx'] = test_df['book_id'].apply(lambda x: item_to_index[x])
    user_item_pairs = test_df[['user_idx', 'book_idx']].values

    model.eval()
    submission = []
    for user, item in user_item_pairs:
        with torch.no_grad():
            user_tensor = torch.tensor([user]).to(device)
            item_tensor = torch.tensor([item]).to(device)
            prediction = model(user_tensor, item_tensor).item()
        submission.append([prediction])
    return submission


submission = predict_missing_values(model, test_df, user_to_index, item_to_index)
#save the submission
submission_df = pd.DataFrame(submission, columns=['rating'])
submission_df.to_csv("submission.csv", index=True)

### Kaggle results

time : 3min

score : 0.88